In [1]:
import os
import pandas as pd
import tabula

In [2]:
def read_budgets(directory):
    budgets = []
    for filename in os.listdir(directory):
        budget_tables = tabula.read_pdf(
            f"{directory}/{filename}", 
            multiple_tables=True
        )
        budgets.append(budget_tables)

    return budgets

In [3]:
# this takes a while
budgets = read_budgets("SY1819_School_Budgets")

In [4]:
# an example list of budgets
sample_budget = budgets[0]
sample_budget

[                     0                  1
 0    Basic Information                NaN
 1     Council District                2nd
 2    Organization Code               1380
 3         School Level  Elementary School
 4         Economically                NaN
 5  Disadvantaged Rate*                NaN
 6                  NaN             83.44%,
               0     1     2               3
 0           NaN  FY14  FY18  FY19 Projected
 1  Enrollment**   842   640             602,
                                                   0            1            2  \
 0                              Position/Expenditure  FY14 Budget  FY18 Budget   
 1                   Principals/Assistant Principals          2.0          1.0   
 2                      Teachers ‐ Regular Education         30.2         25.0   
 3                      Teachers ‐ Special Education          6.0          2.8   
 4      Counselors/Student Adv./ Soc. Serv. Liaisons          1.2          0.8   
 5                         

In [5]:
basic_information = sample_budget[0] #basic information
basic_information

,0,1
0,Basic Information,NaN
1,Council District,2nd
2,Organization Code,1380
3,School Level,Elementary School
4,Economically,NaN
5,Disadvantaged Rate*,NaN
6,NaN,83.44%


In [6]:
def generate_basic_information_table(df):
    '''returns a series representing the "basic information" table'''

    # case for budgets with a comment near the basic information table, e.g. 2050
    if df.shape[1] == 3:
        df = df.iloc[1:, 1:]
        df = df.reset_index(drop=True)
        df = df.T.reset_index(drop=True).T

    # After that, Tabula did pretty well for this table, but didn't get the
    # Economically Disadvanted Rate quite right.

    df.loc[4] = ["Economically Disadvantaged Rate", df.loc[6, 1]]
    df = df.loc[1:4, :]
    return pd.Series(list(df[1]), index=list(df[0]), name='basic_information')

In [7]:
basic_information = generate_basic_information_table(basic_information)
basic_information

Council District                                 2nd
Organization Code                               1380
School Level                       Elementary School
Economically Disadvantaged Rate               83.44%
Name: basic_information, dtype: object

In [8]:
enrollment = sample_budget[1]
enrollment

,0,1,2,3
0,NaN,FY14,FY18,FY19 Projected
1,Enrollment**,842,640,602


In [9]:
def generate_enrollment_table(df):
    '''returns a series representing the "enrollment" table'''
    # nothing too crazy here
    df = df.T.loc[1:, :]
    df_to_series = pd.Series(list(df[1]), index=list(df[0]), name="enrollment")
    return df_to_series.str.replace(',', '').astype(float)

In [10]:
generate_enrollment_table(enrollment)

FY14              842.0
FY18              640.0
FY19 Projected    602.0
Name: enrollment, dtype: float64

In [11]:
operating_funded_allotments = sample_budget[2]
operating_funded_allotments

,0,1,2,3
0,Position/Expenditure,FY14 Budget,FY18 Budget,FY19 Budget
1,Principals/Assistant Principals,2.0,1.0,1.0
2,Teachers ‐ Regular Education,30.2,25.0,24.0
3,Teachers ‐ Special Education,6.0,2.8,5.0
4,Counselors/Student Adv./ Soc. Serv. Liaisons,1.2,0.8,0.1
5,Nurses/Health Services,0.6,1.0,1.0
6,Classroom Assistants/Teacher Assistants,11.0,8.0,9.0
7,Secretaries,1.0,1.0,1.0
8,Support Services Assistants,0.0,2.0,5.0
9,Student Climate Staff,8.0,1.0,3.0


In [12]:
grant_funded_allotments = sample_budget[3]
grant_funded_allotments

,0,1,2,3
0,Position/Expenditure,FY14 Budget,FY18 Budget,FY19 Budget
1,Principals/Assistant Principals,0.0,0.0,0.0
2,Teachers ‐ Regular Education,8.1,8.6,9.6
3,Teachers ‐ Special Education,0.0,0.2,0.0
4,Counselors/Student Adv./ Soc. Serv. Liaisons,0.0,0.2,1.1
5,Nurses/Health Services,0.0,0.0,0.0
6,Classroom Assistants/Teacher Assistants,0.0,0.0,0.0
7,Secretaries,0.0,0.0,0.0
8,Support Services Assistants,7.0,5.0,3.0
9,Student Climate Staff,0.0,7.0,4.0


In [13]:
def generate_allotments_table(df, code, fund):
    '''returns a multiindexed dataframe of org code, fund, and budget category by budget year'''
    df.columns = df.iloc[0]
    df = df.drop(0)
    df = df.set_index(['Position/Expenditure'])
    df = (df.apply(lambda x: x.str.replace('$', '').str.replace(',', ''))
            .astype(float)
          )
    df.name = fund + "ed_allotments"

    df_index_arrays = [
        [code] * len(df),
        [fund] * len(df),
        list(df.index),
    ]

    df.index = pd.MultiIndex.from_arrays(
        df_index_arrays,
        names=("org_code", "fund", "allotment")
    )
    df.columns = [column[:4] for column in df.columns]

    return df

In [14]:
pd.concat([
    generate_allotments_table(
        operating_funded_allotments, "1410", "operating_fund"
    ),
    generate_allotments_table(
        grant_funded_allotments, "1410", "grant_fund"
    )
])

FY14  \
org_code fund           allotment                                                   
1410     operating_fund Principals/Assistant Principals                       2.0   
                        Teachers ‐ Regular Education                         30.2   
                        Teachers ‐ Special Education                          6.0   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          1.2   
                        Nurses/Health Services                                0.6   
                        Classroom Assistants/Teacher Assistants              11.0   
                        Secretaries                                           1.0   
                        Support Services Assistants                           0.0   
                        Student Climate Staff                                 8.0   
                        Other                                                 0.0   
                        Total Positions                                      60.0   
                        Supplies/Equipment/Non Full‐Time Salaries/Other   32272.0   
         grant_fund     Principals/Assistant Principals                       0.0   
                        Teachers ‐ Regular Education                          8.1   
                        Teachers ‐ Special Education                          0.0   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          0.0   
                        Nurses/Health Services                                0.0   
                        Classroom Assistants/Teacher Assistants               0.0   
                        Secretaries                                           0.0   
                        Support Services Assistants                           7.0   
                        Student Climate Staff                                 0.0   
                        Other                                                 1.0   
                        Total Positions                                      16.1   
                        Supplies/Equipment/Non Full‐Time Salaries/Other  198454.0   

                                                                             FY18  \
org_code fund           allotment                                                   
1410     operating_fund Principals/Assistant Principals                       1.0   
                        Teachers ‐ Regular Education                         25.0   
                        Teachers ‐ Special Education                          2.8   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          0.8   
                        Nurses/Health Services                                1.0   
                        Classroom Assistants/Teacher Assistants               8.0   
                        Secretaries                                           1.0   
                        Support Services Assistants                           2.0   
                        Student Climate Staff                                 1.0   
                        Other                                                 1.2   
                        Total Positions                                      43.8   
                        Supplies/Equipment/Non Full‐Time Salaries/Other  100159.0   
         grant_fund     Principals/Assistant Principals                       0.0   
                        Teachers ‐ Regular Education                          8.6   
                        Teachers ‐ Special Education                          0.2   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          0.2   
                        Nurses/Health Services                                0.0   
                        Classroom Assistants/Teacher Assistants               0.0   
                        Secretaries                                           0.0   
                        Support Services Assistants                           5.0

In [15]:
# same data can be derived from the allotments table directly
sample_budget[4]

,0,1,2,3,4
0,NaN,Position/Expenditure,FY14 Budget,FY18 Budget,FY19 Budget
1,Total Positions,NaN,76.1,64.8,67.8
2,Total Supplies/Equipment/Non Full‐Time Salarie...,NaN,"$230,726","$120,136","$112,719"


In [16]:
def generate_all_tables(list_of_df):
    basic_information = generate_basic_information_table(list_of_df[0])
    enrollment = generate_enrollment_table(list_of_df[1])

    operating_funded_allotments = generate_allotments_table(
        list_of_df[2],
        basic_information['Organization Code'],
        'operating_fund'
    )
    grant_funded_allotments = generate_allotments_table(
        list_of_df[3],
        basic_information['Organization Code'],
        'grant_fund'
    )
    operating_and_grant_funded_allotments = pd.concat(
        [operating_funded_allotments, grant_funded_allotments]
    )

    return basic_information, enrollment, operating_and_grant_funded_allotments

In [17]:
basic_information, enrollment, operating_and_grant_funded_allotments = generate_all_tables(sample_budget)

In [18]:
def generate_row(budget_year, basic_information, allotments, enrollment):
    '''returns a Series representing a school's budget for fiscal year.
       budget_year should be FY14, FY18, or FY19'''
 
    
    flattened_allotments = pd.DataFrame(allotments.to_records())
    flattened_allotments.index = flattened_allotments['fund'] +": " + flattened_allotments['allotment']
    flattened_allotments = flattened_allotments.drop(
        ['fund','allotment'], axis=1
    )
    budget_allotments = flattened_allotments[budget_year]
    
    enrollment_label = budget_year + ' Projected' if budget_year == "FY19" else budget_year
    enrollment_index = 'projected_enrollment' if budget_year == "FY19" else 'enrollment'
    enrollment_row = pd.Series(
        enrollment[enrollment_label], index=[enrollment_index]
    )
    
    
    return pd.concat(
            [basic_information,budget_allotments,enrollment_row],
            axis=0
           )

    

In [19]:
generate_row("FY18", basic_information,
             operating_and_grant_funded_allotments, enrollment)

Council District                                                                 2nd
Organization Code                                                               1380
School Level                                                       Elementary School
Economically Disadvantaged Rate                                               83.44%
operating_fund: Principals/Assistant Principals                                    1
operating_fund: Teachers ‐ Regular Education                                      25
operating_fund: Teachers ‐ Special Education                                     2.8
operating_fund: Counselors/Student Adv./ Soc. Serv. Liaisons                     0.8
operating_fund: Nurses/Health Services                                             1
operating_fund: Classroom Assistants/Teacher Assistants                            8
operating_fund: Secretaries                                                        1
operating_fund: Support Services Assistants                      

In [20]:
def generate_tabular_budget(budget_year, budgets):
    '''generate a tabular budget summary for a budget year. Budget year must be FY14,
    FY18, or FY19. Enrollemnt values for budget year 2019 are projected.'''
    school_budget_series = []
    for budget_tables in budgets:
        basic_information, enrollment, operating_and_grant_funded_allotments = generate_all_tables(
            budget_tables
        )
        budget_row = generate_row(
            budget_year, basic_information, operating_and_grant_funded_allotments, enrollment
        )
        budget_row = budget_row
        school_budget_series.append(budget_row)

    return pd.DataFrame(school_budget_series)

In [21]:
fy14 = generate_tabular_budget('FY14', budgets)
fy14['budget_year'] = "FY14"
fy14.to_csv("output/combined_fy14.csv")

fy18 = generate_tabular_budget('FY18', budgets)
fy18['budget_year'] = "FY18"
fy18.to_csv("output/combined_fy18.csv")

fy19 = generate_tabular_budget('FY19', budgets)
fy19['budget_year'] = "FY19"
fy19.to_csv("output/combined_fy19.csv")

In [23]:
combined_tabular_budgets = pd.concat([fy14, fy18, fy19])
combined_tabular_budgets.to_csv("output/all_budgets_tabular.csv")

In [24]:
def generate_hierarchical_budget(budgets):
    school_budgets_dfs = []
    for budget_tables in budgets:
        school_budgets_dfs.append(operating_and_grant_funded_allotments)
    return pd.concat(school_budgets_dfs)

In [25]:
hierarchical_budget = generate_hierarchical_budget(budgets)
hierarchical_budget.to_csv("output/all_budgets_hierarchical.csv")

In [26]:
hierarchical_budget

FY14  \
org_code fund           allotment                                                   
1380     operating_fund Principals/Assistant Principals                       2.0   
                        Teachers ‐ Regular Education                         30.2   
                        Teachers ‐ Special Education                          6.0   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          1.2   
                        Nurses/Health Services                                0.6   
                        Classroom Assistants/Teacher Assistants              11.0   
                        Secretaries                                           1.0   
                        Support Services Assistants                           0.0   
                        Student Climate Staff                                 8.0   
                        Other                                                 0.0   
                        Total Positions                                      60.0   
                        Supplies/Equipment/Non Full‐Time Salaries/Other   32272.0   
         grant_fund     Principals/Assistant Principals                       0.0   
                        Teachers ‐ Regular Education                          8.1   
                        Teachers ‐ Special Education                          0.0   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          0.0   
                        Nurses/Health Services                                0.0   
                        Classroom Assistants/Teacher Assistants               0.0   
                        Secretaries                                           0.0   
                        Support Services Assistants                           7.0   
                        Student Climate Staff                                 0.0   
                        Other                                                 1.0   
                        Total Positions                                      16.1   
                        Supplies/Equipment/Non Full‐Time Salaries/Other  198454.0   
         operating_fund Principals/Assistant Principals                       2.0   
                        Teachers ‐ Regular Education                         30.2   
                        Teachers ‐ Special Education                          6.0   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          1.2   
                        Nurses/Health Services                                0.6   
                        Classroom Assistants/Teacher Assistants              11.0   
...                                                                           ...   
         grant_fund     Secretaries                                           0.0   
                        Support Services Assistants                           7.0   
                        Student Climate Staff                                 0.0   
                        Other                                                 1.0   
                        Total Positions                                      16.1   
                        Supplies/Equipment/Non Full‐Time Salaries/Other  198454.0   
         operating_fund Principals/Assistant Principals                       2.0   
                        Teachers ‐ Regular Education                         30.2   
                        Teachers ‐ Special Education                          6.0   
                        Counselors/Student Adv./ Soc. Serv. Liaisons          1.2   
                        Nurses/Health Services                                0.6   
                        Classroom Assistants/Teacher Assistants              11.0   
                        Secretaries                                           1.0   
                        Support Services Assistants                           0.0   
                        Student Climate Staff                                 8.0 

In [27]:
# this is a nice format for using pandas groupby functions
hierarchical_budget.groupby('allotment').sum()

,FY14,FY18,FY19
allotment,,,
Classroom Assistants/Teacher Assistants,2365.0,1720.0,1935.0
Counselors/Student Adv./ Soc. Serv. Liaisons,258.0,215.0,258.0
Nurses/Health Services,129.0,215.0,215.0
Other,215.0,258.0,215.0
Principals/Assistant Principals,430.0,215.0,215.0
Secretaries,215.0,215.0,215.0
Student Climate Staff,1720.0,1720.0,1505.0
Supplies/Equipment/Non Full‐Time Salaries/Other,49606090.0,25829240.0,24234585.0
Support Services Assistants,1505.0,1505.0,1720.0
